In [1]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np

singlet_bool = True # Set general UCCSD or singlet UCCSD.

bond_len = 0.74#1.45
atom_1 = 'H'
atom_2 = 'H'
basis = 'sto3g'
multiplicity = 1
charge = 0

coordinate_1 = (0.0, 0.0, 0.0)
coordinate_2 = (0.0, 0.0, bond_len)
geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]

molecule = MolecularData(geometry, basis, multiplicity, charge, description='Test')
molecule.load()

ham = get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_q = jordan_wigner(ham)
print(ham_q, '\n')

scf = True      # Hartree-Fock.
mp2 = True      # Moller-Plesset 2.
cisd = True     # Configuration interaction singles and doubles.
ccsd = True     # Coupled cluster singles and doubles.
fci = True      # Full configuration interaction.

calculated_molecule = run_pyscf(molecule, scf, mp2, cisd, ccsd, fci)

if ccsd:
    ccsd_single_amps = calculated_molecule.ccsd_single_amps
    ccsd_double_amps = calculated_molecule.ccsd_double_amps

num_electrons = calculated_molecule.n_electrons
num_qubits = 2*calculated_molecule.n_orbitals

if singlet_bool:
    # Get singlet UCCSD generator.
    packed_amps = uccsd_singlet_get_packed_amplitudes(ccsd_single_amps,  ccsd_double_amps, num_qubits, num_electrons)
    ucc_sing = uccsd_singlet_generator(packed_amps, num_qubits, num_electrons)
    print(ucc_sing)

else:
    # Get general UCCSD operator.
    ucc_op = uccsd_generator(ccsd_single_amps, ccsd_double_amps)
    print(ucc_op)
    
ucc_q = jordan_wigner(ucc_sing)
print(ucc_q)

(-0.48340076819093974+0j) [] +
(-0.05657356483590336+0j) [X0 X1 Y2 Y3] +
(0.05657356483590336+0j) [X0 Y1 Y2 X3] +
(0.05657356483590336+0j) [Y0 X1 X2 Y3] +
(-0.05657356483590336+0j) [Y0 Y1 X2 X3] +
(0.09690774496047531+0j) [Z0] +
(0.1396489174570883+0j) [Z0 Z1] +
(0.08463021714838993+0j) [Z0 Z2] +
(0.1412037819842933+0j) [Z0 Z3] +
(0.09690774496047531+0j) [Z1] +
(0.1412037819842933+0j) [Z1 Z2] +
(0.08463021714838993+0j) [Z1 Z3] +
(-0.04206903445203765+0j) [Z2] +
(0.14735623208609272+0j) [Z2 Z3] +
(-0.04206903445203766+0j) [Z3] 

0.1777989341693255 [0^ 2 1^ 3] +
0.1777989341693255 [1^ 3 0^ 2] +
-0.1777989341693255 [2^ 0 3^ 1] +
-0.1777989341693255 [3^ 1 2^ 0]
0.04444973354233137j [X0 X1 X2 Y3] +
0.04444973354233137j [X0 X1 Y2 X3] +
-0.04444973354233137j [X0 Y1 X2 X3] +
0.04444973354233137j [X0 Y1 Y2 Y3] +
-0.04444973354233137j [Y0 X1 X2 X3] +
0.04444973354233137j [Y0 X1 Y2 Y3] +
-0.04444973354233137j [Y0 Y1 X2 Y3] +
-0.04444973354233137j [Y0 Y1 Y2 X3]


In [2]:
def QubitOperator_to_dict(op, num_qubits):
    
    assert(type(op) == QubitOperator)
    
    op_dict = {}
    
    term_dict = op.terms
    terms = list(term_dict.keys())

    for t in terms:
        
        letters = ['I' for i in range(num_qubits)]

        for i in t:
            letters[i[0]] = i[1]
        
        p_string = ''.join(letters)        
        op_dict[p_string] = term_dict[t]
         
    return op_dict

ham = QubitOperator_to_dict(ham_q, num_qubits)
anz_terms = list((QubitOperator_to_dict(ucc_q, num_qubits)).keys())

In [3]:
print(anz_terms)

['YYYX', 'YXYY', 'XXYX', 'XYYY', 'YXXX', 'YYXY', 'XYXX', 'XXXY']


In [4]:
import cs_vqe_ansatz as c
import cs_vqe as c_org
import numpy as np
from qiskit.opflow.primitive_ops import PauliOp
from qiskit.quantum_info.operators.symplectic.pauli import Pauli
from qiskit.aqua.algorithms import NumPyEigensolver

terms_noncon = c_org.greedy_dfs(ham, 10, criterion='weight')[-1]

#rotations = [[3.1415926536432277, 'ZYX'], [np.pi/4, 'IYI'], [np.pi/4, 'IYZ'], [np.pi/4, 'IIY'], [np.pi/4, 'ZZY']]
rotations = [[np.pi/4, 'YIII'], [np.pi/4, 'YIIZ'], [np.pi/4, 'IYII'], [np.pi/4, 'IYZI'], [np.pi/4, 'IIYI'], [np.pi/4, 'IIYZ']]
#rotations.reverse()

anz = c.construct_ansatz([3], anz_terms, rotations)
print(anz.draw())

#new_terms = ['III', 'ZZX', 'IXI', 'XYI', 'YIX', 'YZI', 'XXX']

#anz = c.construct_ansatz([0, 1, 2], new_terms)
#print(anz.draw())

ham_q = sum([PauliOp(Pauli(k), ham[k]) for k in ham.keys()])
result = NumPyEigensolver(ham_q).run()

exact_result = float(np.real(result.eigenvalues))

vqe_result = c.CS_VQE(ham, terms_noncon, num_qubits=4, ansatz=anz)

print(vqe_result)

           ░ ┌─────┐┌───┐                                         ┌───┐┌───┐ ░ »
q_0: ──────░─┤ SDG ├┤ H ├──■───────────────────────────────────■──┤ H ├┤ S ├─░─»
           ░ ├─────┤├───┤┌─┴─┐                               ┌─┴─┐├───┤├───┤ ░ »
q_1: ──────░─┤ SDG ├┤ H ├┤ X ├──■─────────────────────────■──┤ X ├┤ H ├┤ S ├─░─»
           ░ ├─────┤├───┤└───┘┌─┴─┐                     ┌─┴─┐├───┤├───┤└───┘ ░ »
q_2: ──────░─┤ SDG ├┤ H ├─────┤ X ├──■───────────────■──┤ X ├┤ H ├┤ S ├──────░─»
     ┌───┐ ░ └┬───┬┘└───┘     └───┘┌─┴─┐┌─────────┐┌─┴─┐├───┤└───┘└───┘      ░ »
q_3: ┤ X ├─░──┤ H ├────────────────┤ X ├┤ RZ(2*α) ├┤ X ├┤ H ├────────────────░─»
     └───┘ ░  └───┘                └───┘└─────────┘└───┘└───┘                ░ »
«      ░ ┌─────┐┌───┐                                         ┌───┐┌───┐ ░  ░ »
«q_0: ─░─┤ SDG ├┤ H ├──■───────────────────────────────────■──┤ H ├┤ S ├─░──░─»
«      ░ └┬───┬┘└───┘┌─┴─┐                               ┌─┴─┐├───┤└───┘ ░  ░ »
«q_1: ─░──┤ H ├──────┤ X ├──■──

(-0.9260171757292125, -1.0064869318291205)
